In [1]:
import osmnx as ox
import matplotlib.pyplot as plt
%matplotlib inline

import pickle as pkl

# Specify the name that is used to seach for the data
place_name = "Bangkok, Thailand" #Thailand

# Fetch OSM street network from the location
# graph = ox.graph_from_place(place_name)
graph = pkl.load(open('graph_fix.pkl', 'rb')) 

In [3]:
# Plot the streets
# fig, ax = ox.plot_graph(graph)

In [2]:
area = ox.geocode_to_gdf(place_name) # returns a GeoDataFrame based on the specified place name query

In [3]:
area # check data

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((100.32788 13.80418, 100.32789 13.803...",13.955169,13.216667,100.938604,100.327877,297325349,relation,92277,13.82458,100.622446,"Bangkok, Thailand",boundary,administrative,0.886241


In [ ]:
buildings = ox.geometries_from_place(place_name, tags={'building':True}) # Retrieve buildings from the area:

In [ ]:
len(buildings) # Check how many building footprints we received:

In [ ]:
buildings.head(3) # Check the first rows:

In [ ]:
# Retrieve restaurants
restaurants = ox.geometries_from_place(place_name, 
                                  tags={"amenity": "restaurant"}
                                 )
# How many restaurants do we have?
len(restaurants)

In [11]:
# Fetch Bangkok street network
graph = ox.graph_from_place("Bangkok, Thailand")
# Get statistic of graph (number of nodes, edges, etc.)
ox.basic_stats(graph)

{'n': 183151,
 'm': 398051,
 'k_avg': 4.3466975337289995,
 'edge_length_total': 34457749.39399898,
 'edge_length_avg': 86.56616713435962,
 'streets_per_node_avg': 2.310148456737883,
 'streets_per_node_counts': {0: 0,
  1: 70441,
  2: 461,
  3: 97331,
  4: 14847,
  5: 64,
  6: 7},
 'streets_per_node_proportions': {0: 0.0,
  1: 0.3846061446565948,
  2: 0.002517048773962468,
  3: 0.5314248898449913,
  4: 0.0810642584534073,
  5: 0.0003494384415045509,
  6: 3.8219829539560254e-05},
 'intersection_count': 112710,
 'street_length_total': 18817546.060999963,
 'street_segment_count': 211550,
 'street_length_avg': 88.95082042543116,
 'circuity_avg': 1.0538989100172231,
 'self_loop_proportion': 0.001366107303238005}

In [15]:
edges = list(graph.edges())

In [16]:
edge = edges[0]

In [23]:
graph.get_edge_data(edge[0], edge[1])

{0: {'osmid': 202235939,
  'name': 'ถนนบรรทัดทอง',
  'highway': 'tertiary',
  'oneway': False,
  'reversed': False,
  'length': 61.174}}

In [16]:
# use %%timeit to see how long it takes to run the function

11.3 ns ± 0.0512 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [6]:
# Get all nodes in graph
nodes = list(graph.nodes())

In [64]:
# Random 2 nodes for try shortest path algorithm
import random
orig = random.choice(nodes)
dest = random.choice(nodes)

In [65]:
# using networkx's dijkstra algorithm to find shortest path (from orig to dest)

In [66]:
%%timeit
shortest_path = nx.shortest_path(graph, orig, dest, weight='length')

449 ms ± 16.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
%%timeit
shortest_path_length = nx.shortest_path_length(graph, orig, dest, weight='length')

486 ms ± 24.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
# Define the heuritic function
from math import cos, sqrt
def heuristic(u, v):
    u_point = (graph.nodes[u]['y'], graph.nodes[u]['x'])
    v_point = (graph.nodes[v]['y'], graph.nodes[v]['x'])
    
    coords_1 = u_point
    coords_2 = v_point

    R = 6371000
    conversion_const = 0.0174533

    c_1 = (coords_1[0]*conversion_const,coords_1[1]*conversion_const)
    c_2 = (coords_2[0]*conversion_const,coords_2[1]*conversion_const)


    delta_phi = abs(c_1[1]-c_2[1])

    theta = c_1[0]
    delta_theta = abs(c_1[0]-c_2[0])

    del_x = R*cos(theta)*delta_phi 
    del_y = R*delta_theta

    ans = sqrt(del_x**2+del_y**2)
    return ans*2

In [69]:
# using networkx's A-star algorithm to find shortest path (from orig to dest) with heuristic funtion

In [70]:
%%timeit
astar_path = nx.astar_path(graph, orig, dest, heuristic=heuristic, weight='length')

7.76 ms ± 350 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [71]:
%%timeit
astar_path_length = nx.astar_path_length(graph, orig, dest, heuristic=heuristic, weight='length')

8.04 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
import time
import numpy as np
import networkx as nx
import random
nodes = list(graph.nodes())

In [4]:
G = pkl.load(open('bangkok_graph.pkl', 'rb'))

Mean duration: 0.45 seconds
Median duration: 0.48 seconds
Standard deviation: 0.26 seconds


In [8]:
n = 100
i = 0
durations = []
while i<n:
    start_time = time.time()
    orig = random.choice(nodes)
    dest = random.choice(nodes)
    try:
        astar_path_length = nx.astar_path_length(graph, orig, dest, heuristic=heuristic, weight='length')
    except:
        continue
    end_time = time.time()
    duration = end_time - start_time
    durations.append(duration)
    i += 1
duration_mean = np.mean(durations)
duration_median = np.median(durations)
duration_std =  np.std(durations)
print("Mean duration: {:.2f} seconds".format(duration_mean))
print("Median duration: {:.2f} seconds".format(duration_median))
print("Standard deviation: {:.2f} seconds".format(duration_std))
    

Mean duration: 0.03 seconds
Median duration: 0.01 seconds
Standard deviation: 0.04 seconds


In [4]:
import random

In [14]:
# code for find u,v that not have path
count = 0
while count < 20:
    orig = random.choice(nodes)
    dest = random.choice(nodes)
    try:
        astar_path_length = nx.astar_path_length(graph, orig, dest, heuristic=heuristic, weight='length')
    except:
        count += 1
        with open("err_nodes.txt", "a") as f:
            f.write(str(orig) + ":" + str(graph.nodes()[orig]) + ", " + str(dest) + ":" + str(graph.nodes()[dest]) + "\n")

In [10]:
orig

3011050475

In [13]:
graph.nodes()[orig]

{'y': 13.7703336, 'x': 100.399658, 'street_count': 3}

In [15]:
graph

In [16]:
g = ox.get_undirected(graph)

In [19]:
pkl.dump(g, open("bangkok_graph_drive_undir.pkl", "wb"))

In [23]:
nodes = list(g.nodes())

In [24]:
# code for find u,v that not have path
count = 0
while count < 20:
    orig = random.choice(nodes)
    dest = random.choice(nodes)
    try:
        astar_path_length = nx.astar_path_length(g, orig, dest, heuristic=heuristic, weight='length')
    except:
        count += 1
        with open("err_nodes.txt", "a") as f:
            f.write(str(orig) + ":" + str(g.nodes()[orig]) + ", " + str(dest) + ":" + str(g.nodes()[dest]) + "\n")

In [21]:
g

In [25]:
g.edges(orig, dest)

MultiEdgeDataView([(6004358296, 6004358297, None)])